#### Copyright 2021 Google LLC.

SPDX-License-Identifier: Apache-2.0

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Linear Regression

This assignment introduces linear regression with gradient descent, but starts with basic code building blocks.

As you read through this notebook, you'll some mathematical notation that may not all be familiar. Don't panic! We will go through all of this in more detail in class and lab.

## Overview

###Learning Objectives
* Practice manipulating numpy arrays.
* Generate artifical data, which can be useful for building intuition and debugging issues with complex models.
* Use Mean Squared Error to compare models.
* Understand generalization and overfitting.
* Work through the derivation of gradient descent for linear regression.
* Apply gradient descent to build intuition about training models.

### Grading

This assignment (exercises 1-8) is worth a total of 40 points.

## Artificial Data

As a warm-up, let's get some practice manipulating matrices with numpy, plotting with matplotlib, and computing evaluation metrics for some simple models.

In [ ]:
# Import the libraries we'll use below.
import numpy as np
import matplotlib.pyplot as plt

## Data as matrices
Data usually comes in the form of matrices. The Python Numpy library makes it easy to manipulate matrices (*arrays* in numpy) efficiently. See the [Numpy Tutorial](https://docs.scipy.org/doc/numpy/user/quickstart.html) for details.

A few important details to point out:

* Numpy arrays follow the (row, column) indexing convention so the shape of a matrix with $r$ rows and $c$ columns is $(r,c)$.
* Remember that matrix multiplication requires the inner dimensions to align.
* A 1-dimensional array (*vector*) with $n$ elements has shape $(n,)$ and transposing it has no effect.

In [ ]:
# Print these to make sure you understand what is being generated.
A = np.array([1, 2, 3])
B = np.arange(1, 13).reshape(3, 4)
C = np.ones((2, 3))
D = np.eye(3)

### Exercise 1: Matrix manipulation (7pts)

Perform the following computations using numpy functions and print the results. Each can easily be expressed in a single line of code -- please use Numpy operations rather than for loops.

\begin{align}
1. && 2A + 1 \\
2. && \textrm{Sum the rows of } B \\
3. && \textrm{Sum the columns of } B \\
4. && \textrm{Number of elements of $B$ greater than 5} \\
5. && C + C \\
6. && A * B \\
7. && (B * B^{\textrm{transpose}}) - D \\
\end{align}

#### Student Solution

In [ ]:
# YOUR CODE HERE

# 1
2*A + 1

In [ ]:
# 2
np.sum(B, axis=1)

In [ ]:
# 3
np.sum(B, axis=0)

In [ ]:
# 4
np.sum(B >5)

In [ ]:
# 5
C + C

In [ ]:
# 6
np.dot(A,B)



In [ ]:
# 7
np.dot(B,B.T) - D

## Data for Supervised Learning
Supervised learning is all about learning to make predictions: given an input $x$ (e.g. home square footage), can we predict an output $\hat{y}$ (e.g. estimated value) as close to the actual observed output $y$ (e.g. sale price) as possible. Note that the "hat" above $y$ is used to denote an estimated or predicted value. The *output* is sometimes referred to as a *target* or a *label*.

Let's start by generating some artificial data. We'll create a vector of inputs, $X$, and a corresponding vector of target outputs $Y$. In general, we'll refer to invidual examples with a lowercase ($x$), and a vector or matrix containing multiple examples with a capital ($X$).

In [ ]:
def create_1d_data(num_examples=10, w=2, b=1, random_scale=1):
  """Create X, Y data with a linear relationship with added noise.

  Args:
    num_examples: number of examples to generate
    w: desired slope
    b: desired intercept
    random_scale: add uniform noise between -random_scale and +random_scale

  Returns:
    X and Y with shape (num_examples)
  """
  X = np.arange(num_examples)
  np.random.seed(4)  # consistent random number generation
  noise = np.random.uniform(low=-random_scale, high=random_scale, size=X.shape)
  Y = w * X + b + noise
  return X, Y

In [ ]:
# Create some artificial data using create_1d_data.
X, Y = create_1d_data()
plt.scatter(X, Y)
plt.show()

### Exercise 2: Models for Data (3 pts)
A model is a function that takes an input $x$ and produces a prediction $\hat{y}$.

Let's consider two possible models for this data:
1. $M_1(x) = x+5 = \hat{y^1}$
2. $M_2(x) = 2x+1 = \hat{y^2}$

Compute the predictions of models $M_1$ and $M_2$ for the values in $X$. These predictions should be vectors of the same shape as $Y$. Then plot the prediction lines of these two models overlayed on the "observed" data $(X, Y)$. Use [plt.plot()](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html) to draw the lines.

Which of these models is better?

#### Student Solution

In [ ]:
# YOUR CODE HERE

def model_1(x):
  return x + 5

def model_2(x):
  return (2*x)+1

model_1_output = model_1(X)
model_2_output = model_2(X)


plt.xlabel("Input x")
plt.ylabel("Prediction y")
plt.title("Predictions: Model 1 vs Model 2")
plt.scatter(X, Y, color = "red", label = "Observed Data")
plt.plot(model_1_output, color = "green", label = "Model 1")
plt.plot(model_2_output, color = "blue", label = "Model 2")
plt.legend()
plt.show()

Model 2 is better since the line of best fit it produces through the observed data is closer to what is expected

## Evaluation Metrics

How good are our models? Intuitively, the better the model, the more closely it fits the data we have. That is, for each $x$, we'll compare $y$, the true value, with $\hat{y}$, the predicted value. This comparison is often called the *loss* or the *error*. One common such comparison is *squared error*: $(y-\hat{y})^2$. Averaging over all our data points, we get the *mean squared error*:

\begin{equation}
\textit{MSE} = \frac{1}{|Y|} \sum_{y_i \in Y}(y_i - \hat{y_i})^2
\end{equation}

This is a very common way to evaluate models and heavily punishes predictions that are far away from the actual value. Linear Regression models are learned by optimizing $\textit{MSE}$.

### Exercise 3: Computing MSE (3 pts)
Write a function for computing the MSE metric and use it to compute the MSE for the two models above, $M_1$ and $M_2$. You should be able to do this **without using a for loop**.

#### Student Solution

In [ ]:
# YOUR CODE HERE
def MSE(true_values, predicted_values):
  """Return the MSE between true_values and predicted values."""

  if len(true_values) != len(predicted_values):
        raise ValueError("Dimensions mistmatch")

  # Calculate the mean squared error
  mse = np.mean((true_values - predicted_values) ** 2)
  return mse


In [ ]:
#Computing the MSE for Model 1:
model_1_mse = MSE(Y, model_1_output)
print("Model 1 MSE: ",model_1_mse)

# Computing MSE for Model 2:
model_2_mse = MSE(Y, model_2_output)
print("Model 2 MSE: ", model_2_mse)

## Generalization

Our data $(X, Y)$ represents just a sample of all possible input-output pairs we might care about. A model will be useful to the extent we can apply it to new inputs. Consider the more complex model below, which appears to produce a much smaller mean squared error.

In [ ]:
# Fit a 7-th degree polynomial to (X, Y). See np.polyfit for details.
polynomial_model_coefficients = np.polyfit(X, Y, deg=7)
polynomial_model = np.poly1d(polynomial_model_coefficients) # creates a one-dimensional polynomial model
M3 = polynomial_model(X)
X_plot = np.linspace(0, 10, 100) # generates points to draw the polynomial curve
M3_plot = polynomial_model(X_plot)
fig = plt.scatter(X, Y)
plt.plot(X_plot, M3_plot, '-k')
print ('MSE for M3:', MSE(Y, M3))

However, we care about how well a model will apply to data we have not yet seen. Typically we do this by having a **training set**, which the models learns from, and a **test set**, which the model is evaluated on. You want to ensure that your model does not **overfit** to the training set, capturing infromation that does not generalize well.

In general, ML practitioners try not to use `np.polyfit` models that have high degrees, as those do not generalize well. Especially for $X$ values that fall outside the range of the training data.

### Exercise 4: Generalization (4 pts)
Apply models M2 and M3 to the test data below. Print the predicted outputs of the models and compare the MSE between the predictions and true values (Y_test).

In [ ]:
# Suppose we have the following test data:
X_test = np.array([10, 11, 12])
Y_test = np.array([22.1, 23.4, 26.2])

# Plot train and test data.
plt.scatter(X, Y, color='k', label='Train')
plt.scatter(X_test, Y_test, color='r', label='Test')
plt.legend()
plt.show()

#### Student Solution

In [ ]:
# YOUR CODE HERE
# Model 2
model_2_test_data = model_2(X_test)
model_2_test_mse = MSE(Y_test,model_2_test_data)

print("Model 2 prediction test data: ", model_2_test_data)
print("Model 2 test data MSE: ", model_2_test_mse)

#Model 3
print("")
model_3_test_data = polynomial_model(X_test)
model_3_test_mse = MSE(Y_test,model_3_test_data)

print("Model 3 prediction test data: ", model_3_test_data)
print("Model 3 test data MSE: ", model_3_test_mse)


## Notation
In our artificial data, things are pretty simple: each input example is just a single value. But soon, each input example will include multiple values or *features*, so we need some conventions to avoid confusion.

Let's start with the inputs:

\begin{align}
X =
\begin{pmatrix}
x_0 \\
x_1 \\
\vdots \\
x_{m-1}
\end{pmatrix}
\end{align}

* Capital $X$ refers to all input examples together.
* Lowercase $x$ refers to an individual input example; we use $x_i$ to refer to input example $i$; there are $m$ total examples.

Further, each input example $x$ could itself be a vector of feature values:

\begin{align}
x_i = [x_{i,0}, x_{i,1}, \dots x_{i,n-1}]
\end{align}

* Lowercase $x$ refers to all input features together for an individual input example.
* $x_i$ refers to feature $i$ for an input example $x$; there are $n$ total features.

For example, $x_i$ could be an individual person. Then, $x_{i,j}$ is some attribute about the person like height. $X$ is the matrix of all people,.

Similarly, we can index labels $y_i$ in $Y$, which we can think of as a column vector where $y_i$ is the label for $x_i$.

\begin{align}
Y =
\begin{pmatrix}
y_0 \\
y_1 \\
\vdots \\
y_{m-1}
\end{pmatrix}
\end{align}

From now on, we will be using matrix notation by default. Rows refer to examples and columns refer to features. If we want to be very specific and refer to a particular feature of a particular input example, we can use $x_{i,j}$ for input $i$, feature $j$. Using matrices will be useful for coding ML algorithms since most of the operations we will do can be expressed as operations on matrices.


##Parameter Vectors
Let's prepare to learn a linear model $M(x)$ that approximates values of $Y$ from corresponding values of $X$. Since our input data has only one feature, our model will have two parameters (also called weights), which we'll refer to collectively as $W$:

\begin{align}
M(x) = w_0 + w_1x
\end{align}

If we use the model from our data generation (`create_1d_data`), $w_1=2$ and $w_0=1$.

We can make this more concise. Notice that, if we prepend an extra feature (column) to $X$ that is always $1$, we can rewrite our model using a matrix multiplication:

\begin{align}
M(x) = w_0x_0 + w_1x_1 = xW^T
\end{align}

To make this matrix formulation as clear as possible, this is:

\begin{align}
\hat{y} = xW^T =
\begin{pmatrix}
x_0 & x_1 \\
\end{pmatrix}
\begin{pmatrix}
w_0 \\
w_1 \\
\end{pmatrix}
\end{align}

In addition, if we wanted to apply our model to *all* inputs $X$, we could simply use $XW^T$:

\begin{align}
\hat{Y} = XW^T =
\begin{pmatrix}
x_{0,0} & x_{0,1} \\
x_{1,0} & x_{1,1} \\
\vdots & \vdots \\
x_{m-1,0} & x_{m-1,1} \\
\end{pmatrix}
\begin{pmatrix}
w_0 \\
w_1 \\
\end{pmatrix}
\end{align}

Note that this notation does not care about number of features - if we were to add more features to $X$ and more weights to $W$, this notation would still hold up.

Remember that [matrix multiplication](https://en.wikipedia.org/wiki/Matrix_multiplication) requires the inner dimensions to line up:

\begin{align}
X_{\{m \times n\}} W^T_{\{n \times 1 \}}  = \hat{Y}_{\{m \times 1 \}}
\end{align}

### Exercise 5: Practice with Parameters (5 pts)

Rewrite $M_1$ and $M_2$ (from exercise 2) using the above notation.

This will required adding a column of $1$s to $X$ and using matrix multiplication (`np.dot`).

Assert that your new values match the previous predictions, and that the shape of your predictions also match the shape of your labels $Y$.

#### Student Solution

In [ ]:
# Add a column of 1s to X by using np.c_ to concatenate with the current values.
X_with_1s = np.c_[np.ones(X.shape[0]), X]
# YOUR CODE HERE
def M1(x):
  W = np.array([5,1])
  return np.dot(x,W)

def M2(x):
  W = np.array([1,2])
  return np.dot(x,W)

print("Model 1 new: ",M1(X_with_1s))
print("Model 1 old: ", model_1(X))
print("")
print("Model 2 new: ",M2(X_with_1s))
print("Model 2 old: ",model_2(X))


## Gradient Descent
Here, we'll demonstrate gradient descent for linear regression to learn the weight vector $W$. We'll use the more specific notation $M_W(x)$ since we want to specify that $M$ is parameterized by $W$. As above, we'll assume that $x_0=1$ so we can write $M$ as a sum or a matrix product:

\begin{align}
M_W(x) = \hat{Y} = \sum_{i=0}^{n-1} w_i x_i = x W^T
\end{align}

To make the math more readable the derivation that follows, we'll use summations. However, when we implement this in the code below, we'll use matrix computations.

In linear regression, we compute the loss, $J(W)$ from the mean squared difference between predictions $M_W(x)$ and targets $y$. In the following equation, we average the loss over each of the $m$ training examples.

\begin{align}
J(W) = \frac{1}{2m} \sum_{i=0}^{m-1} (M_W(x^{(i)}) - y^{(i)})^2
\end{align}

You'll notice that, unlike in our $\textit{MSE}$ calculation above, we divide by $2$ here. Dividing by $2$ simplifies the formula of the gradient, since it cancels out the constant $2$ from by the derivative of the squared term (see below). Because error is relative to other models' errors, it does not matter the we divide by $2$ as long as we do it to every error.

Remember that the gradient is a vector of partial derivatives for each $w_j$ (holding the other elements of $w$ constant). The gradient points in direction of steepest ascent for the loss function $J$.

Here we derive the parameter update rule by computing the gradient of the loss function. We need a derivative for each feature in $x$, so we'll show how to compute the derivative with respect to $w_j$. For simplicity, let's assume we have only one training example ($m = 1$):

\begin{align}
\frac{\partial}{\partial w_j} J(W) &= \frac{\partial}{\partial w_j} \frac{1}{2} (M_W(x) - y)^2 \tag{1}\\
&= 2 \cdot \frac{1}{2} (M_W(x) - y) \cdot \frac{\partial}{\partial w_j} (M_W(x) - y)\tag{2} \\
&= (M_W(x) - y) \frac{\partial}{\partial w_j} \left(\sum_{i=0}^{n-1} w_i x_i - y_i\right)\tag{3} \\
&= (M_W(x) - y)x_j\tag{4}
\end{align}

The derivation has 2 key steps:

(1) Apply the [chain rule](https://en.wikipedia.org/wiki/Chain_rule) (step 1 -> 2).

(2) The derivative with respect to $w_j$ of $M_W(x) - y$ is only non-zero for $w_j x_j$; $y_i$ does not depend on $w_j$, nor do any values $w_i x_i$ where $i \neq j$. For this component, the derivative is $x_j$ (step 2 -> 4).

Ok, that's it. We can now implement gradient descent for linear regression. The only difference in the code below is that it computes the loss as an average over all training examples (rather than just a single example).

### Exercise 6: Implementing Gradient Descent for Linear Regression (8 pts)
Fill in the `NotImplemented` parts of the gradient descent function below. There are detailed comments to help guide you. Note that this function uses vectors and matrices so you'll want to use numpy functions like `np.dot` instead of loops.

Hint: print the arrays or array shape if you are not sure how array multiplication works.

##### Student Solution

In [ ]:
def gradient_descent(inputs, outputs, learning_rate, num_epochs):
  """Apply the gradient descent algorithm to learn learn linear regression.

  Args:
    inputs: A 2-D array where each column is an input feature and each
            row is a training example.
    outputs: A 1-D array containing the real-valued
             label corresponding to the input data in the same row.
    learning_rate: The learning rate to use for updates.
    num_epochs: The number of passes through the full training data.

  Returns:
    weights: A 2-D array with the learned weights after each training epoch.
    losses: A 1-D array with the loss after each epoch.
  """
  # m = number of examples, n = number of features
  m, n = inputs.shape

  # We'll use a vector of size n to store the learned weights and initialize
  # all weights to 1.
  W = np.ones(n)

  # Keep track of the training loss and weights after each step.
  losses = []
  weights = []

  for epoch in range(num_epochs):
      # Append the old weights to the weights list to keep track of them.
      weights.append(W)

      # Evaluate the current predictions for the training examples given
      # the current estimate of W (you did this in exercise 5).
      predictions = np.matmul(inputs, W.T)
      # Find the difference between the predictions and the actual targetvalues.
      diff = predictions - outputs

      # In standard linear regression, we want to minimize the sum of squared
      # differences. Find the loss as we did in Excerise 3, but use the extra
      # scaling factor (as in Formula (1) above, and discussed in class).
      loss = 0.5*((predictions - outputs)**2).mean()

      # Append the loss to the losses list to keep a track of it.
      losses.append(loss)

      # Compute the gradient with respect to the loss.
      # [Formula (4) in the Gradient Descent Implementation]
      # Hint: There are m training samples. We need to compute the loss as an
      # average over all training examples.
      gradient = np.dot((predictions - outputs), inputs) / m

      # Update weights, scaling the gradient by the learning rate.
      W = W - learning_rate * gradient

  return np.array(weights), np.array(losses)

Let's try running gradient descent with our artificial data and print out the results. Note that we're passing the version of the input data with a column of $1s$ so that we learn an *intercept* (also called a *bias*). We can also try learning without the intercept.

Note: if your implementation of gradient descent is correct, you should get a loss of ~0.20458 after 5 epochs (with a bias parameter).

In [ ]:
print('Running gradient descent...')
weights, losses = gradient_descent(X_with_1s, Y, learning_rate=.02,
                                   num_epochs=5)
for W, loss in zip(weights, losses):
  print(loss, W)

print('\nRunning gradient descent without biases...')
# Make sure we're providing an input with the right 2-D shape.
X_without_1s = np.expand_dims(X, axis=0).T
weights_without_bias, losses_without_bias = gradient_descent(X_without_1s, Y,
                                                             .02, num_epochs=5)
for W, loss in zip(weights_without_bias, losses_without_bias):
  print(loss, W)

### Exercise 7: Interpreting the Model (5 pts)
Write down the learned model with and without an intercept term. Which model fits the data better? How are you sure?

#### Student Solution

1. Model with intercept term:
$M(x) = w$<sub>0</sub>$ + w$<sub>1</sub>$x$ = 1.16277124 + 1.93072732$x$

2. Model without intercept term:
$M(x)=w$<sub>1</sub>$x$ = 2.10371403$x$


The model with the intercept fits the data better. From the ouputs of the gradient descent, after 5 interations it is evident that the model with the bias has the lower value. Which means our model with the bias reflect more closely the actual (truth) data than the model without the bias.

## Gradient Descent Progress
Let's write a function that lets us visualize the progress of gradient descent during training. Our gradient descent function already provides intermediate weight vectors and losses after each epoch, so we just need to plot these.

In [ ]:
def plot_learning(inputs, outputs, weights, losses):
  """Plot predictions and losses after each training epoch.

  Args:
    inputs: A 2-D array where each column is an input feature and each
            row is a training example.
    outputs: A 1-D array containing the real-valued
             label corresponding to the input data in the same row.
    weights: A 2-D array with the learned weights after each training epoch.
    losses: A 1-D array with the loss after each epoch.
  """
  # Create a figure.
  plt.figure(1, figsize=[10,4])

  # The first subplot will contain the predictions. Start by plotting the
  # outputs (Y).
  plt.subplot(121)
  plt.xlabel('x')
  plt.ylabel('y')
  plt.xticks(inputs[:,1])
  plt.scatter(inputs[:,1], outputs, color='black', label='Y')

  # For each epoch, retrieve the estimated weights W, compute predictions, and
  # plot the resulting line.
  num_epochs = len(weights)
  for i in range(num_epochs):
    W = weights[i]
    predictions = np.dot(inputs, W.T)
    plt.plot(inputs[:,1], predictions, label='Epoch %d' %i)
  plt.legend()

  # The second subplot will contain the losses.
  plt.subplot(122)
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.xticks(range(num_epochs))
  plt.plot(range(num_epochs), losses, marker='o', color='black',
           linestyle='dashed')
  plt.show()

### Exercise 8: Plotting Progress (5 pts)

Re-run gradient descent using X_with_1s, but this time with learning_rate=0.01 and num_epochs=7.

Run the plot_learning function using the weights and losses returned by gradient_descent (from above) and answer the following questions:

1. Is learning converging faster or slower with `learning rate=0.01` than when we used `learning_rate=0.02`?
2. If you continue training, will the loss eventually reach 0? Why?
3. If you continue training, will the model eventually converge to $h(x)=2x+1$? Why?

#### Student Solution

In [ ]:
weights, losses = gradient_descent(X_with_1s, Y, learning_rate=.01,num_epochs=7)
plot_learning(X_with_1s, Y, weights, losses)

print("\n\n")
for W, loss in zip(weights, losses):
  print(loss, W)

In [ ]:
#Plotting the graphs with learning rate 0.02 for reference
weights, losses = gradient_descent(X_with_1s, Y, learning_rate=.02,num_epochs=7)
plot_learning(X_with_1s, Y, weights, losses)

WRITE YOUR ANSWERS HERE

1. It is converging slower with a learning rate of 0.01

2. No the loss would not reach zero. We are using a line of best fit to find an approximation for the prediction. In this paritcular model, the data does not create a perfect straight line, (data potentially contains noise) hence we would never truly reach a loss of zero, though we may get close to it.

3. No it would not. The data does not form a perfect straight line, this noise in the data makes it very challenging for it to converge at a particular equation since there will potentially always be some value for the MSE. Also, I ran the model for a great number of epochs, (approximately 100k) and found that the w<sub>0</sub> value approximates to 1.777.. and the w<sub>1</sub> value approximates to 1.862... the trend was that both w<sub>0</sub> and w<sub>1</sub> were increasing. I also plotted the graphs for both of these models (2x + 1) and (1.862x + 1.778) and found that the two lines were converging but after a certain point start to diverge. (See graph below)



In [ ]:
x = np.linspace(0, 10, 100)  # Creating 100 evenly spaced points between -10 and 10

# Calculate y values using the equation y = 2x + 1
y = 2 * x + 1
y_2 = 1.862*(x) + 1.777

# Plot the function
plt.plot(x, y, label='y = 2x + 1')
plt.plot(x, y_2, label='y = 1.862x + 1.777')
# Add labels and title
plt.xlabel('x')
plt.ylabel('y')
plt.title('Plot of y = 2x + 1')

plt.grid(True)
plt.legend()
plt.show()